In [22]:
import pandas as pd 

#função criada para coletar as bases de dados e transformar em dataframe
def criar_dataframe(caminho_arquivo1, caminho_arquivo2):
    coletas_solicitadas = pd.read_excel(caminho_arquivo1)
    apuracao_lucratividade = pd.read_excel(caminho_arquivo2)
    return coletas_solicitadas, apuracao_lucratividade
    
#passando os caminhos das bases de dados
caminho_arquivo1,caminho_arquivo2 = "COLETAS_SOLICITADAS_BFS1.XLSX","APURACAO_LUCRATIVIDADE_BFS1.xlsx"


#definindo os dataframe extraindo das bases de dados
coletas_solicitadas,apuracao_lucratividade = criar_dataframe(caminho_arquivo1,caminho_arquivo2)

def format_coletas_solicitadas(dataframe):
    #Excluindo a primeira linha do dataframe
    dataframe.drop([0],axis=0, inplace=True)
    #criando o dataframe com o numero do ct e o numero de romaneio
    dataframe2 = dataframe[["ROMANEIO","NUMERO"]]
    #utilizando a função (replace, para trocar os dados pela função , pd.NA e assim transformar os dados em NaN
    #e utilizando a função dropna(para remover todas os dados NaN)
    dataframe2 = dataframe2.replace("0.0", pd.NA).dropna(subset=["NUMERO","ROMANEIO"])
    #retornando o dataframe2
    return dataframe2
    
#chamando a função e adicionando o dataframe a uma nova variavel
coletas_solicitadas2 = format_coletas_solicitadas(coletas_solicitadas)


#exibindo o dataframe
#display(coletas_solicitadas2)

def assimilar_valores_coletas(dataframe):
    #esta linha de código, cria um novo dataframe, e utiliza da função groupby para agrupar os numeros com base no romaneio e transformar em lista
    coleta_solicitadas_agrupado = dataframe.groupby("NUMERO")["ROMANEIO"].apply(list).reset_index()
    #utilizando a expressão lambda pora remover os [colchestes] dos valores da coluna romaneio
    coleta_solicitadas_agrupado["ROMANEIO"] = coleta_solicitadas_agrupado["ROMANEIO"].apply(lambda x: x[0])
    #formatando os valores na coluna NUMERO e ROMANEIO
    coleta_solicitadas_agrupado['ROMANEIO'] = coleta_solicitadas_agrupado['ROMANEIO'].astype(int)
    coleta_solicitadas_agrupado["NUMERO"] = coleta_solicitadas_agrupado["NUMERO"].astype(int)
    #Criando um novo dataframe e renomeando as colunas
    coleta_solicitadas_agrupado = coleta_solicitadas_agrupado[["ROMANEIO","NUMERO"]]
    coleta_solicitadas_agrupado.rename(columns={"NUMERO":"ORDEM COLETA"}, inplace=True)
    #definindo a coluna romaneio com "str" e adicionando sempre 6 numero ao valor das celulas
    coleta_solicitadas_agrupado['ROMANEIO'] = coleta_solicitadas_agrupado['ROMANEIO'].astype(str)
    coleta_solicitadas_agrupado['ROMANEIO'] = coleta_solicitadas_agrupado['ROMANEIO'].str.zfill(6)
    #salvando o arquivo
    #coleta_solicitadas_agrupado.to_excel("teste1.xlsx")
    return coleta_solicitadas_agrupado

#criando um novo dataframe com base no resultado da função formatando a base de dados
coleta_solicitadas_agrupado = assimilar_valores_coletas(coletas_solicitadas2)
#display(coleta_solicitadas_agrupado)



def format_apuracao_lucratividade(dataframe):
    #criando outro dataframe com base nas, FILIAL, SERIE, CTE
    apuracao_lucratividade2 = apuracao_lucratividade[["Unnamed: 5","Unnamed: 6","Unnamed: 7"]]
    #renomeando as colunas extraidas
    apuracao_lucratividade2.rename(columns={"Unnamed: 5":"FILIAL","Unnamed: 6":"SERIE","Unnamed: 7":"CTE"},inplace=True)
    #concatenando as colunas "ordem coleta" e "CTE" e formando um novo dataframe
    apuracao_lucratividade3 = pd.concat([coleta_solicitadas_agrupado["ORDEM COLETA"], apuracao_lucratividade2["CTE"]], axis=1)
    #Assimilando os valores da coluna "ordem coleta" em "cte"
    apuracao_lucratividade4 = apuracao_lucratividade3.groupby("ORDEM COLETA")["CTE"].apply(list).reset_index()
    #removendo os "[colchetes]" do valores das celulas 
    apuracao_lucratividade4["CTE"] = apuracao_lucratividade4["CTE"].apply(lambda x: x[0])
    #adicionando o valor "0" a celula da linha 0 da coluna "CTE", para substituir o valor "NaN" adicionando pela função
    apuracao_lucratividade4.at[0,"CTE"] = 0
    #seprando a coluna "romaneio" e criando um novo dataframe
    apuracao_lucratividade5 = coleta_solicitadas_agrupado[["ROMANEIO"]]
    #concatenando dos dataframes de, "ordem coleta, cte" com o "romaneio"
    apuracao_lucratividade6 = pd.concat([apuracao_lucratividade5, apuracao_lucratividade4],axis=1)
    #transformando os dados de "ordem coleta, cte" em inteiros
    novos_tipos_de_dados = {'CTE': int,"ORDEM COLETA":int}
    apuracao_lucratividade6 = apuracao_lucratividade6.astype(novos_tipos_de_dados)
    #apuracao_lucratividade6.to_excel("PRE_FINALIZADO_MARCO_BFS1.xlsx")
    apuracao_lucratividade_copia = apuracao_lucratividade.copy()
    
    #retornando o dataframe fortamdo
    return apuracao_lucratividade6, apuracao_lucratividade_copia

#chamando a função e passando o dataframe para obter as formatações    
apuracao_lucratividade6, apuracao_lucratividade_copia = format_apuracao_lucratividade(apuracao_lucratividade)
#exebindo o dataframe
#display(apuracao_lucratividade6)

def finalizar_dataframe(caminho1,caminho2):
    #criar o dataframe com base nos dados oferecidos
    finalizado1 = caminho1
    #remover a 1 linha do 1 dataframe
    #finalizado1.drop(["Unnamed: 0"],axis=1,inplace=True)
    # ----------------------------------------------------
    #criando dataframe para ser concatenado
    apuracao = caminho2
    #concatenando eles
    finalizado2 = pd.concat([apuracao,finalizado1],axis=1)
    #excluindo colunas
    finalizado2.drop(["CONTROLE","CTE"],axis=1,inplace=True)
    #renomeando as colunas
    finalizado2.rename(columns={"Unnamed: 5":"FILIAL","Unnamed: 6":"SERIE","Unnamed: 10":"ORIGEM CIDADE","Unnamed: 12":"CIDADE DESTINO","Unnamed: 7":"CTE"},inplace=True)
    #função utilizada para transformar todas as linhas do dataframe que estão no formato "NaN" para "0"
    finalizado2 = finalizado2.fillna(0)
   
    #transformando as colunas especificadas no tipo inteiro e atribuindo 6 valores a coluna romaneio
    finalizado2["ROMANEIO"] = finalizado2["ROMANEIO"].astype(int)
    finalizado2["ORDEM COLETA"] = finalizado2["ORDEM COLETA"].astype(int)
    finalizado2["SERIE"] = finalizado2["SERIE"].astype(int)
    finalizado2["CTE"] = finalizado2["CTE"].astype(int)
    finalizado2['ROMANEIO'] = finalizado2['ROMANEIO'].astype(str)
    finalizado2['ROMANEIO'] = finalizado2['ROMANEIO'].str.zfill(6)
   
     #criando uma lista para ree-ordenar as colunas e executando isto
    troca_colunas = ["MANIFESTO","MOTORISTAS","DATA EMISSAO", "ROMANEIO","ORDEM COLETA","DOC EMBARQUE","FILIAL","SERIE","CTE","PAGADOR"
                     ,"ORIGEM","ORIGEM CIDADE","DESTNO","CIDADE DESTINO","KGS", "FRETE","KGS COLETA","CUSTO COLETA","CUSTO TRANSFERENCIA","CUSTO ENTREGA","RCF","RCTRC","FLUVIAL",
                     "IMPOSTO", "TOTAL CUSTO","SALDO"]
    finalizado2 = finalizado2[troca_colunas]
    
    #salvando o novo dataframe em formato de excel
    finalizado2.to_excel("FINALIZADO_MARÇO_BFS1.xlsx")
    #retornando o dataframe concluído
    return finalizado2
#caminhos dos dataframes a serem utilizados 
caminho1 = apuracao_lucratividade6
caminho2 = apuracao_lucratividade_copia

#extraindo o dataframe concluido
finalizado =  finalizar_dataframe(caminho1,caminho2)

#exibindo o dataframe
display(finalizado)

C:\Users\brito\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\brito\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ROMANEIO,NUMERO
3,1281.0,297.0
4,1290.0,304.0
5,1308.0,319.0
6,1314.0,328.0
9,1285.0,300.0
10,1313.0,327.0
12,132.0,317.0
16,1301.0,21153.0
17,1291.0,305.0
19,1306.0,316.0


,ROMANEIO,ORDEM COLETA
0,001281,297
1,001285,300
2,001290,304
3,001291,305
4,001306,316
5,000132,317
6,001308,318
7,001308,319
8,001313,327
9,001314,328


C:\Users\brito\AppData\Local\Temp\ipykernel_10468\183197983.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apuracao_lucratividade2.rename(columns={"Unnamed: 5":"FILIAL","Unnamed: 6":"SERIE","Unnamed: 7":"CTE"},inplace=True)


,ROMANEIO,ORDEM COLETA,CTE
0,001281,297,0
1,001285,300,288
2,001290,304,289
3,001291,305,290
4,001306,316,291
5,000132,317,292
6,001308,318,293
7,001308,319,294
8,001313,327,295
9,001314,328,296


,MANIFESTO,MOTORISTAS,DATA EMISSAO,ROMANEIO,ORDEM COLETA,DOC EMBARQUE,FILIAL,SERIE,CTE,PAGADOR,...,KGS COLETA,CUSTO COLETA,CUSTO TRANSFERENCIA,CUSTO ENTREGA,RCF,RCTRC,FLUVIAL,IMPOSTO,TOTAL CUSTO,SALDO
0,0,0,0,001281,297,0,0,0,0,0,...,0.000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00
1,BFS/FSA-000230/24,PAULO CEZAR SILVA DE,2024-03-01 00:00:00,001285,300,CT,BFS,1,288,ACOS VITAL COMERCIO DE TUBOS HIDRAULICOS EIRELI,...,0.000,0.00,0.00,600.00,1.66,2.05,0.0,0.00,603.71,662.75
2,BFS/FSA-000233/24/BFS/FSA-000236/24,GILMARCOS ALVES DOS,2024-03-01 00:00:00,001290,304,CT,BFS,1,289,U&M MINERACAO E CONSTRUCAO S.A,...,299.100,0.00,66.44,105.15,12.37,15.26,0.0,0.00,199.22,504.10
3,0,0,2024-03-04 00:00:00,001291,305,CT,BFS,1,290,ELETROSANDRO LTDA,...,0.000,0.00,0.00,0.00,4.50,5.55,0.0,0.00,10.05,1389.95
4,FSA/BHZ-003538/24/BMG/BSP-000072/24,WILLSANDER MACIEL BA,2024-03-04 00:00:00,001306,316,CT,BFS,1,291,DYNATRONIX INDUSTRIAL LTDA,...,0.000,0.00,3038.64,0.00,0.63,1.45,0.0,0.00,3040.72,-1456.34
5,BFS/SLM-000231/24,WILLSANDER MACIEL BA,2024-03-04 00:00:00,000132,317,CT,BFS,1,292,MAGNESITA MINERACAO S.A.,...,12.090,0.00,0.00,2.51,0.36,0.45,0.0,0.00,3.32,135.44
6,BFS/SLM-000231/24,WILLSANDER MACIEL BA,2024-03-04 00:00:00,001308,318,CT,BFS,1,293,MAGNESITA MINERACAO S.A.,...,0.000,0.00,0.00,0.15,0.02,0.03,0.0,0.00,0.20,117.67
7,BFS/FSA-000233/24/BFS/FSA-000236/24,GILMARCOS ALVES DOS,2024-03-05 00:00:00,001308,319,CT,BFS,1,294,U&M MINERACAO E CONSTRUCAO S.A,...,185.480,120.81,15.85,25.08,2.64,3.26,0.0,0.00,167.64,-2.58
8,BFS/FSA-000235/24,CARLOS CLEONE MACHAD,2024-03-06 00:00:00,001313,327,CT,BFS,1,295,GERMANO PNEUS LTDA,...,0.000,0.00,0.00,0.00,2.68,3.31,0.0,0.00,5.99,831.36
9,BFS/FSA-000235/24,CARLOS CLEONE MACHAD,2024-03-06 00:00:00,001314,328,CT,BFS,1,296,GERMANO PNEUS LTDA,...,0.000,0.00,0.00,0.00,1.74,2.15,0.0,0.00,3.89,840.29


In [23]:
import pandas as pd 

#função criada para coletar as bases de dados e transformar em dataframe
def criar_dataframe(caminho_arquivo1, caminho_arquivo2):
    coletas_solicitadas = pd.read_excel(caminho_arquivo1)
    apuracao_lucratividade = pd.read_excel(caminho_arquivo2)
    return coletas_solicitadas, apuracao_lucratividade
    
#passando os caminhos das bases de dados
caminho_arquivo1,caminho_arquivo2 = "COLETAS_SOLICITADAS_BFS1.XLSX","APURACAO_LUCRATIVIDADE_BFS1.xlsx"


#definindo os dataframe extraindo das bases de dados
coletas_solicitadas,apuracao_lucratividade = criar_dataframe(caminho_arquivo1,caminho_arquivo2)

def format_coletas_solicitadas(dataframe):
    #Excluindo a primeira linha do dataframe
    dataframe.drop([0],axis=0, inplace=True)
    #criando o dataframe com o numero do ct e o numero de romaneio
    dataframe2 = dataframe[["ROMANEIO","NUMERO"]]
    #utilizando a função (replace, para trocar os dados pela função , pd.NA e assim transformar os dados em NaN
    #e utilizando a função dropna(para remover todas os dados NaN)
    dataframe2 = dataframe2.replace("0.0", pd.NA).dropna(subset=["NUMERO","ROMANEIO"])
    #retornando o dataframe2
    return dataframe2
    
#chamando a função e adicionando o dataframe a uma nova variavel
coletas_solicitadas2 = format_coletas_solicitadas(coletas_solicitadas)


#exibindo o dataframe
#display(coletas_solicitadas2)

def assimilar_valores_coletas(dataframe):
    #esta linha de código, cria um novo dataframe, e utiliza da função groupby para agrupar os numeros com base no romaneio e transformar em lista
    coleta_solicitadas_agrupado = dataframe.groupby("NUMERO")["ROMANEIO"].apply(list).reset_index()
    #utilizando a expressão lambda pora remover os [colchestes] dos valores da coluna romaneio
    coleta_solicitadas_agrupado["ROMANEIO"] = coleta_solicitadas_agrupado["ROMANEIO"].apply(lambda x: x[0])
    #formatando os valores na coluna NUMERO e ROMANEIO
    coleta_solicitadas_agrupado['ROMANEIO'] = coleta_solicitadas_agrupado['ROMANEIO'].astype(int)
    coleta_solicitadas_agrupado["NUMERO"] = coleta_solicitadas_agrupado["NUMERO"].astype(int)
    #Criando um novo dataframe e renomeando as colunas
    coleta_solicitadas_agrupado = coleta_solicitadas_agrupado[["ROMANEIO","NUMERO"]]
    coleta_solicitadas_agrupado.rename(columns={"NUMERO":"ORDEM COLETA"}, inplace=True)
    #definindo a coluna romaneio com "str" e adicionando sempre 6 numero ao valor das celulas
    coleta_solicitadas_agrupado['ROMANEIO'] = coleta_solicitadas_agrupado['ROMANEIO'].astype(str)
    coleta_solicitadas_agrupado['ROMANEIO'] = coleta_solicitadas_agrupado['ROMANEIO'].str.zfill(6)
    #salvando o arquivo
    #coleta_solicitadas_agrupado.to_excel("teste1.xlsx")
    return coleta_solicitadas_agrupado

#criando um novo dataframe com base no resultado da função formatando a base de dados
coleta_solicitadas_agrupado = assimilar_valores_coletas(coletas_solicitadas2)
#display(coleta_solicitadas_agrupado)



def format_apuracao_lucratividade(dataframe):
    #criando outro dataframe com base nas, FILIAL, SERIE, CTE
    apuracao_lucratividade2 = apuracao_lucratividade[["Unnamed: 5","Unnamed: 6","Unnamed: 7"]]
    #renomeando as colunas extraidas
    apuracao_lucratividade2.rename(columns={"Unnamed: 5":"FILIAL","Unnamed: 6":"SERIE","Unnamed: 7":"CTE"},inplace=True)
    #concatenando as colunas "ordem coleta" e "CTE" e formando um novo dataframe
    apuracao_lucratividade3 = pd.concat([coleta_solicitadas_agrupado["ORDEM COLETA"], apuracao_lucratividade2["CTE"]], axis=1)
    #Assimilando os valores da coluna "ordem coleta" em "cte"
    apuracao_lucratividade4 = apuracao_lucratividade3.groupby("ORDEM COLETA")["CTE"].apply(list).reset_index()
    #removendo os "[colchetes]" do valores das celulas 
    apuracao_lucratividade4["CTE"] = apuracao_lucratividade4["CTE"].apply(lambda x: x[0])
    #adicionando o valor "0" a celula da linha 0 da coluna "CTE", para substituir o valor "NaN" adicionando pela função
    apuracao_lucratividade4.at[0,"CTE"] = 0
    #seprando a coluna "romaneio" e criando um novo dataframe
    apuracao_lucratividade5 = coleta_solicitadas_agrupado[["ROMANEIO"]]
    #concatenando dos dataframes de, "ordem coleta, cte" com o "romaneio"
    apuracao_lucratividade6 = pd.concat([apuracao_lucratividade5, apuracao_lucratividade4],axis=1)
    #transformando os dados de "ordem coleta, cte" em inteiros
    novos_tipos_de_dados = {'CTE': int,"ORDEM COLETA":int}
    apuracao_lucratividade6 = apuracao_lucratividade6.astype(novos_tipos_de_dados)
    #apuracao_lucratividade6.to_excel("PRE_FINALIZADO_MARCO_BFS1.xlsx")
    apuracao_lucratividade_copia = apuracao_lucratividade.copy()
    
    #retornando o dataframe fortamdo
    return apuracao_lucratividade6, apuracao_lucratividade_copia

#chamando a função e passando o dataframe para obter as formatações    
apuracao_lucratividade6, apuracao_lucratividade_copia = format_apuracao_lucratividade(apuracao_lucratividade)
#exebindo o dataframe
#display(apuracao_lucratividade6)

def finalizar_dataframe(caminho1,caminho2):
    #criar o dataframe com base nos dados oferecidos
    finalizado1 = caminho1
    #remover a 1 linha do 1 dataframe
    #finalizado1.drop(["Unnamed: 0"],axis=1,inplace=True)
    # ----------------------------------------------------
    #criando dataframe para ser concatenado
    apuracao = caminho2
    #concatenando eles
    finalizado2 = pd.concat([apuracao,finalizado1],axis=1)
    #excluindo colunas
    finalizado2.drop(["CONTROLE","CTE"],axis=1,inplace=True)
    #renomeando as colunas
    finalizado2.rename(columns={"Unnamed: 5":"FILIAL","Unnamed: 6":"SERIE","Unnamed: 10":"ORIGEM CIDADE","Unnamed: 12":"CIDADE DESTINO","Unnamed: 7":"CTE"},inplace=True)
    #função utilizada para transformar todas as linhas do dataframe que estão no formato "NaN" para "0"
    finalizado2 = finalizado2.fillna(0)
   
    #transformando as colunas especificadas no tipo inteiro e atribuindo 6 valores a coluna romaneio
    finalizado2["ROMANEIO"] = finalizado2["ROMANEIO"].astype(int)
    finalizado2["ORDEM COLETA"] = finalizado2["ORDEM COLETA"].astype(int)
    finalizado2["SERIE"] = finalizado2["SERIE"].astype(int)
    finalizado2["CTE"] = finalizado2["CTE"].astype(int)
    finalizado2['ROMANEIO'] = finalizado2['ROMANEIO'].astype(str)
    finalizado2['ROMANEIO'] = finalizado2['ROMANEIO'].str.zfill(6)
   
     #criando uma lista para ree-ordenar as colunas e executando isto
    troca_colunas = ["MANIFESTO","MOTORISTAS","DATA EMISSAO", "ROMANEIO","ORDEM COLETA","DOC EMBARQUE","FILIAL","SERIE","CTE","PAGADOR"
                     ,"ORIGEM","ORIGEM CIDADE","DESTNO","CIDADE DESTINO","KGS", "FRETE","KGS COLETA","CUSTO COLETA","CUSTO TRANSFERENCIA","CUSTO ENTREGA","RCF","RCTRC","FLUVIAL",
                     "IMPOSTO", "TOTAL CUSTO","SALDO"]
    finalizado2 = finalizado2[troca_colunas]
    
    #salvando o novo dataframe em formato de excel
    finalizado2.to_excel("FINALIZADO_MARÇO_BFS1.xlsx")
    #retornando o dataframe concluído
    return finalizado2
#caminhos dos dataframes a serem utilizados 
caminho1 = apuracao_lucratividade6
caminho2 = apuracao_lucratividade_copia

#extraindo o dataframe concluido
finalizado =  finalizar_dataframe(caminho1,caminho2)

#exibindo o dataframe
display(finalizado)

C:\Users\brito\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\brito\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\brito\AppData\Local\Temp\ipykernel_10468\935194093.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apuracao_lucratividade2.rename(columns={"Unnamed: 5":"FILIAL","Unnamed: 6":"SERIE","Unnamed: 7":"CTE"},inplace=True)


,MANIFESTO,MOTORISTAS,DATA EMISSAO,ROMANEIO,ORDEM COLETA,DOC EMBARQUE,FILIAL,SERIE,CTE,PAGADOR,...,KGS COLETA,CUSTO COLETA,CUSTO TRANSFERENCIA,CUSTO ENTREGA,RCF,RCTRC,FLUVIAL,IMPOSTO,TOTAL CUSTO,SALDO
0,0,0,0,001281,297,0,0,0,0,0,...,0.000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00
1,BFS/FSA-000230/24,PAULO CEZAR SILVA DE,2024-03-01 00:00:00,001285,300,CT,BFS,1,288,ACOS VITAL COMERCIO DE TUBOS HIDRAULICOS EIRELI,...,0.000,0.00,0.00,600.00,1.66,2.05,0.0,0.00,603.71,662.75
2,BFS/FSA-000233/24/BFS/FSA-000236/24,GILMARCOS ALVES DOS,2024-03-01 00:00:00,001290,304,CT,BFS,1,289,U&M MINERACAO E CONSTRUCAO S.A,...,299.100,0.00,66.44,105.15,12.37,15.26,0.0,0.00,199.22,504.10
3,0,0,2024-03-04 00:00:00,001291,305,CT,BFS,1,290,ELETROSANDRO LTDA,...,0.000,0.00,0.00,0.00,4.50,5.55,0.0,0.00,10.05,1389.95
4,FSA/BHZ-003538/24/BMG/BSP-000072/24,WILLSANDER MACIEL BA,2024-03-04 00:00:00,001306,316,CT,BFS,1,291,DYNATRONIX INDUSTRIAL LTDA,...,0.000,0.00,3038.64,0.00,0.63,1.45,0.0,0.00,3040.72,-1456.34
5,BFS/SLM-000231/24,WILLSANDER MACIEL BA,2024-03-04 00:00:00,000132,317,CT,BFS,1,292,MAGNESITA MINERACAO S.A.,...,12.090,0.00,0.00,2.51,0.36,0.45,0.0,0.00,3.32,135.44
6,BFS/SLM-000231/24,WILLSANDER MACIEL BA,2024-03-04 00:00:00,001308,318,CT,BFS,1,293,MAGNESITA MINERACAO S.A.,...,0.000,0.00,0.00,0.15,0.02,0.03,0.0,0.00,0.20,117.67
7,BFS/FSA-000233/24/BFS/FSA-000236/24,GILMARCOS ALVES DOS,2024-03-05 00:00:00,001308,319,CT,BFS,1,294,U&M MINERACAO E CONSTRUCAO S.A,...,185.480,120.81,15.85,25.08,2.64,3.26,0.0,0.00,167.64,-2.58
8,BFS/FSA-000235/24,CARLOS CLEONE MACHAD,2024-03-06 00:00:00,001313,327,CT,BFS,1,295,GERMANO PNEUS LTDA,...,0.000,0.00,0.00,0.00,2.68,3.31,0.0,0.00,5.99,831.36
9,BFS/FSA-000235/24,CARLOS CLEONE MACHAD,2024-03-06 00:00:00,001314,328,CT,BFS,1,296,GERMANO PNEUS LTDA,...,0.000,0.00,0.00,0.00,1.74,2.15,0.0,0.00,3.89,840.29
